<a href="https://colab.research.google.com/github/Manasaoruganti/-Compliance-Risk-Tracker-dashboard/blob/main/Rooftop_Analysis_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the datasets (Excel files)
buildings_df = pd.read_excel('/content/Buildings data.xlsx')
sites_df = pd.read_excel('/content/Sites data.xlsx')

# Select relevant columns from Buildings data and rename for consistency
buildings_cols = ['SITE NAME', 'TOTAL ROOFTOP AREA AVAILABLE IN (sq.m)',
                  'TOTAL FEASIBLE AREA FOR RTS INSTALLATION IN (sq.m)',
                  'TOTAL FEASIBLE CAPACITY FOR RTS INSTALLATION (kW)',
                  'CURRENT STATUS OF RTS INSTALLATION - INSTALLED CAPACITY IN (kW)']
buildings_df_selected = buildings_df[buildings_cols].copy()
buildings_df_selected.rename(columns={'TOTAL ROOFTOP AREA AVAILABLE IN (sq.m)': 'TOTAL ROOFTOP AREA AVAILABLE (sq.m)'}, inplace=True)

# Select relevant columns from Sites data and rename for consistency
sites_cols = ['State', 'SITE NAME', 'SANCTIONED LOAD (kW)']
sites_df_selected = sites_df[sites_cols].copy()
sites_df_selected.rename(columns={'State': 'state', 'SANCTIONED LOAD (kW)': 'sanctioned load'}, inplace=True)

# Merge the two dataframes
# Use outer merge to keep all records and fill missing values as needed, although left merge would also work given the problem description.
merged_df = pd.merge(buildings_df_selected, sites_df_selected, on='SITE NAME', how='left')

# Calculate 'No. of Buildings of that site'
num_buildings_per_site = merged_df.groupby('SITE NAME').size().reset_index(name='No. of Buildings of that site')

# Calculate 'No. of Buildings of the particular site in particular state'
num_buildings_per_site_state = merged_df.groupby(['state', 'SITE NAME']).size().reset_index(name='no of buildings of the perticular site in perticular state')

# Group by 'state' and 'SITE NAME' and aggregate data
final_df = merged_df.groupby(['state', 'SITE NAME']).agg(
    # Sum 'TOTAL ROOFTOP AREA AVAILABLE (sq.m)' for 'Total Rooftop area' per site
    Total_Rooftop_area=('TOTAL ROOFTOP AREA AVAILABLE (sq.m)', 'sum'),
    # Use first for 'Sanctioned Load' because it should be constant per site
    Sanctioned_Load=('sanctioned load', 'first'),
    Total_Feasible_Area_RTS_Installation_sq_m=('TOTAL FEASIBLE AREA FOR RTS INSTALLATION IN (sq.m)', 'sum'),
    Total_Feasible_Capacity_RTS_Installation_kW=('TOTAL FEASIBLE CAPACITY FOR RTS INSTALLATION (kW)', 'sum'),
    Current_Status_RTS_Installation_Installed_Capacity_kW=('CURRENT STATUS OF RTS INSTALLATION - INSTALLED CAPACITY IN (kW)', 'sum')
).reset_index()

# Merge the building counts back into the final_df
final_df = pd.merge(final_df, num_buildings_per_site, on='SITE NAME', how='left')
final_df = pd.merge(final_df, num_buildings_per_site_state, on=['state', 'SITE NAME'], how='left')


# Calculate Tentative Capacity: Compare the sanctioned load and feasible capacity and take the min of both
# As assumed, 'feasible area' in the user's prompt meant 'TOTAL FEASIBLE CAPACITY FOR RTS INSTALLATION (kW)'
final_df['Tentative Capacity(Compare the sanctioned load and feasible area and take the min of both )'] = \
    final_df[['Sanctioned_Load', 'Total_Feasible_Capacity_RTS_Installation_kW']].min(axis=1)

# Rename columns to match desired output headers
final_df = final_df.rename(columns={
    'SITE NAME': 'SITE name of perticular state',
    'Total_Rooftop_area': 'Total Rooftop area',
    'Sanctioned_Load': 'Sanctioned Load',
    'Total_Feasible_Area_RTS_Installation_sq_m': 'TOTAL FEASIBLE AREA FOR RTS INSTALLATION IN (sq.m)',
    'Total_Feasible_Capacity_RTS_Installation_kW': 'TOTAL FEASIBLE CAPACITY FOR RTS INSTALLATION (kW)',
    'Current_Status_RTS_Installation_Installed_Capacity_kW': 'CURRENT STATUS OF RTS INSTALLATION - INSTALLED CAPACITY IN (kW)'
})

# Reorder columns to match the requested output
output_columns = [
    'state',
    'SITE name of perticular state',
    'No. of Buildings of that site',
    'Total Rooftop area',
    'Sanctioned Load',
    'TOTAL FEASIBLE AREA FOR RTS INSTALLATION IN (sq.m)',
    'Tentative Capacity(Compare the sanctioned load and feasible area and take the min of both )',
    'TOTAL FEASIBLE CAPACITY FOR RTS INSTALLATION (kW)',
    'CURRENT STATUS OF RTS INSTALLATION - INSTALLED CAPACITY IN (kW)',
    'no of buildings of the perticular site in perticular state'
]

final_df = final_df[output_columns]

# Save the final DataFrame to a new CSV file
output_filename = 'RTS_Installation_Summary.csv'
final_df.to_csv(output_filename, index=False)

print(f"Data successfully processed and saved to {output_filename}")

Data successfully processed and saved to RTS_Installation_Summary.csv
